<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/breaker_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [114]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import FactorAnalysis
import joblib

In [115]:
#load models if needed
scaler = joblib.load('trivium_scaler.pkl')
pca= joblib.load('trivium_pca.pkl')

In [116]:
#load files
trivium= pd.read_csv('trivium_scorecard.csv')
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)
webscraping.keys()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


dict_keys(['Events and8', 'Events', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

In [117]:
#Load battle winners
#threefold
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
#Trivium
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
#traditional
traditional = webscraping['Battles and8 Traditional']
traditional.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
traditional[['Event', 'Category']] = traditional[['Event', 'Category']].fillna(method='ffill')

In [118]:
tr=traditional[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
th=threefold[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
tv=trivium[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()

dataframes = [('traditional', tr), ('threefold', th), ('trivium', tv)]

# Create an empty list to store modified dataframes
appended_dfs = []

# Iterate to create a combined dataframe
for name, df in dataframes:
    df['Original_DF'] = name
    df['Battle_ID'] = df.index
    appended_dfs.append(df)
appendedBattles = pd.concat(appended_dfs, ignore_index=True)
appendedBattles

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID
0,Championnat de France 2023,B-Boys +16 Top 16 Round Robin Group D,Kidmario,Noé,Noé,traditional,0
1,Championnat de France 2023,B-Boys +16 Top 16 Round Robin Group D,Wilfried,Killian,Killian,traditional,1
2,Championnat de France 2023,B-Boys +16 Top 16 Round Robin Group D,Wilfried,Noé,Noé,traditional,2
3,Championnat de France 2023,B-Boys +16 Top 16 Round Robin Group D,Kidmario,Killian,Killian,traditional,3
4,Championnat de France 2023,B-Boys +16 Top 16 Round Robin Group D,Wilfried,Kidmario,Wilfried,traditional,4
...,...,...,...,...,...,...,...
2624,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,Rerunn,Rerunn,trivium,1521
2625,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,Rerunn,Rerunn,trivium,1522
2626,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Piccolo,D-Matt,D-Matt,trivium,1523
2627,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,trivium,1524


In [119]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

Events = webscraping['Events'][1:].copy()
new_header = Events.iloc[0]
Events = Events.drop(1)
Events.columns = new_header
Events.rename(columns={'Name':'Event'}, inplace=True)
Events.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF, Events], ignore_index=True)

In [120]:
Events

1,Date,Event,Category,Stage,Link,Format,Scraped
2,2023-06-07 00:00:00,Campionato Italiano di Categoria Breaking FIDS...,NaN,NaN,NaN,NaN,NaN
3,2023-10-14 00:00:00,Hip Hop Grand Slam & DTV Ranking Battle 2023,NaN,NaN,NaN,NaN,NaN
4,2023-06-23 00:00:00,UK BBOY CHAMPIONSHIPS / UNDISPUTED LONDON 2023,NaN,NaN,NaN,NaN,NaN
5,2023-04-06 00:00:00,WDSF BfG International Series - Montreal - Can...,NaN,NaN,NaN,NaN,NaN
6,2029-11-30 00:00:00,Castle Rock Jam & Battle - Vol. 2 2019,NaN,NaN,NaN,NaN,NaN
7,2020-01-18 00:00:00,Castle Rock Jam & Battle - Vol. 3 2020,NaN,NaN,NaN,NaN,NaN
8,2023-07-05 00:00:00,Championnat de France 2023,NaN,NaN,NaN,NaN,NaN
9,2020-04-04 00:00:00,Groove E Session 2020,NaN,NaN,NaN,NaN,NaN
10,2017-12-09 00:00:00,Hustle & Freeze – Vol. 12 2017,NaN,NaN,NaN,NaN,NaN
11,2023-11-25 00:00:00,Path to Flow Jam - Vol. I 2017,NaN,NaN,NaN,NaN,NaN


In [121]:
#add battle date
appendedBattles=pd.merge(appendedBattles, combined_events[['Date', 'Event']], on='Event', how='left')
